In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

#import time
#import argparse
#import multiprocessing


from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from IPython.display import display

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

C:\ProgramData\Anaconda3\envs\tensorflow_final\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow_final\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow_final\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow

In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [4]:
MODEL_NAME = 'inference_graph/ssdlite_mobilenet_v2_coco_58282'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('annotations', 'labelmap-all.pbtxt')

In [5]:
NUM_CLASSES = 5

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES)
category_index = label_map_util.create_category_index(categories)

In [6]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # Processamento de uma única imagem
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # É necessário atualizar a imagem para converter a máscara das coordenadas da caixa em coordenadas da imagem e ajustar o tamanho da imagem.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')
    #print(image_tensor.eval, "\n")

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})
    #print(output_dict, "\n")
    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [8]:
#import glob

import pandas as pd

#targetPattern = r"CSGO_images\test\*.jpg"
#arquivos = glob.glob(targetPattern)

pasta = r"CSGO_images//test//"
fileExt = r".jpg"
fileExt_1 = r".JPG"
arquivos  = [_ for _ in os.listdir(pasta) if _.endswith(fileExt) or _.endswith(fileExt_1)]
arquivos

['1outa1.jpg',
 '1outa100.jpg',
 '1outa112.jpg',
 '1outa153.jpg',
 '1outa167.jpg',
 '1outa174.jpg',
 '1outa180.jpg',
 '1outa185.jpg',
 '1outa20.jpg',
 '1outa211.jpg',
 '1outa227.jpg',
 '1outa238.jpg',
 '1outa239.jpg',
 '1outa265.jpg',
 '1outa27.jpg',
 '1outa270.jpg',
 '1outa288.jpg',
 '1outa293.jpg',
 '1outa300.jpg',
 '1outa32.jpg',
 '1outa323.jpg',
 '1outa358.jpg',
 '1outa376.jpg',
 '1outa381.jpg',
 '1outa410.jpg',
 '1outa421.jpg',
 '1outa44.jpg',
 '1outa444.jpg',
 '1outa447.jpg',
 '1outa448.jpg',
 '1outa452.jpg',
 '1outa454.jpg',
 '1outa460.jpg',
 '1outa462.jpg',
 '1outa513.jpg',
 '1outa524.jpg',
 '1outa539.jpg',
 '1outa557.jpg',
 '1outa56.jpg',
 '1outa573.jpg',
 '1outa580.jpg',
 '1outa584.jpg',
 '1outa592.jpg',
 '1outa605.jpg',
 '1outa608.jpg',
 '1outa61.jpg',
 '1outa615.jpg',
 '1outa620.jpg',
 '1outa680.jpg',
 '1outa710.jpg',
 '1outa711.jpg',
 '1outa716.jpg',
 '1outa724.jpg',
 '1outa735.jpg',
 '1outa740.jpg',
 '1outa749.jpg',
 '1outa75.jpg',
 '1outa751.jpg',
 '1outa772.jpg',
 '1out

In [9]:
test_csv = pd.read_csv('test_labels_1.csv')
arr = np.array(test_csv)
arr

array([['1outa1.jpg', 2, 0.38888888899999996, 0.484375,
        0.7517361109999999, 0.59765625],
       ['1outa100.jpg', 2, 0.42708333299999995, 0.436523438,
        0.7795138890000001, 0.6171875],
       ['1outa112.jpg', 2, 0.43576388899999996, 0.47265625, 0.84375,
        0.5859375],
       ...,
       ['IMG_9365.JPG', 0, nan, nan, nan, nan],
       ['IMG_9366.JPG', 0, nan, nan, nan, nan],
       ['IMG_9367.JPG', 0, nan, nan, nan, nan]], dtype=object)

In [10]:
import cv2
previsoes = []
l = 0

Verdadeiro_positivo = 0
Falso_positivo = 0
Falso_negativo = 0
Verdadeiro_negativo = 0

for w in arquivos:
    image_np = cv2.imread(pasta + w)
    print("step:", l)
    l = l + 1 
    try:
        with detection_graph.as_default():
                with tf.compat.v1.Session() as sess:
                    
                    # Get handles to input and output tensors
                    ops = tf.compat.v1.get_default_graph().get_operations()
                    all_tensor_names = {output.name for op in ops for output in op.outputs}
                    tensor_dict = {}
                    
                    for key in [
                      'num_detections', 'detection_boxes', 'detection_scores',
                      'detection_classes', 'detection_masks'
                    ]:
                        tensor_name = key + ':0'
                        if tensor_name in all_tensor_names:
                            tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
                          tensor_name)
                            #print(tensor_dict[key], "\n")

                    #ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    #print("image_np_expanded", "\n")
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    #print(output_dict, "\n")

                    test_result = tf.greater(output_dict['detection_scores'], [0.50])
                    j = 0

                    linha = []
                    previsoes_2 = []
                    linha.append(w)
                    
                    
                    for i in test_result.eval():
                        if i == True:
                            linha = []
                            linha.append(w)
                            linha.append(output_dict['detection_classes'][j])
                            linha.append(output_dict['detection_boxes'][j][0])
                            linha.append(output_dict['detection_boxes'][j][1])
                            linha.append(output_dict['detection_boxes'][j][2])
                            linha.append(output_dict['detection_boxes'][j][3])
                            #print(linha_1, "\n")
                            
                            #linha.append(linha_1)
                            #linha.append(linha_2)

                            previsoes.append(linha)
                            previsoes_2.append(linha)
                            print(linha, "\n")
                            
                        j = j + 1
                        
                    index = np.argwhere(arr == w)
                    print("index")
                    print("1", index)
                    print(len(index))
                    print("arr1", arr[index[0][0]][1])

                    if len(previsoes_2) != 0:       
                        
                        matrix_resultado = []
                        q = []
                        n = 0
                        for i in index:
                            k = []
                            m = 0
                            for g in previsoes_2:
                                Erro_cord = []
                                #print("if2", arr[i[0]][1] == g[1], "identificação de classe", arr[i[0]][1], g[1])
                                if arr[i[0]][1] == g[1]:
                                    Erro_cord.append(arr[i[0]][2] - g[2])
                                    Erro_cord.append(arr[i[0]][3] - g[3])
                                    Erro_cord.append(arr[i[0]][4] - g[4])
                                    Erro_cord.append(arr[i[0]][5] - g[5])
                                    erro_1 = tf.greater(Erro_cord, [0.05])
                                    erro_2 = tf.less(Erro_cord, [-0.05])
                                    with tf.compat.v1.Session() as sess:
                                        if any(o == True for o in erro_1.eval()) or any(o == True for o in erro_2.eval()):
                                            #print("Coordenada errada")
                                            k.append(False)
                                            
                                        else:                           
                                            #print("acetou")
                                            k.append(True)
                                            
                                else:
                                    #print("Classe errada")
                                    k.append(False)
                                m = m + 1        
                            q.append(k)
                            print("k", k)
                                
                            n = n + 1
                        print("q", q)
                        print("m", m)
                        print("n", n)
                        
                        df = pd.DataFrame(q)
                        for f in range(0, m, 1):
                            print("F1", f)
                            c = np.all(df[f] == False)
                            print("Coluna", f, c)
                            if c:
                                Falso_positivo = Falso_positivo + 1
                        
                        for f in range(0, n, 1):
                            print("F2", f)
                            c = np.any(q[f])
                            print("Linha", f, c)
                            if c:
                                Verdadeiro_positivo = Verdadeiro_positivo + 1
                            else:
                                Falso_negativo = Falso_negativo + 1
                    
                    else:
                        if arr[index[0][0]][1] == 0:
                            Verdadeiro_negativo = Verdadeiro_negativo + 1
                    
                        else:                        
                            Falso_negativo = Falso_negativo + 1
                            #print("não encontrou nada", "Falso_negativo", Falso_negativo)
                    
                    print("Falso_negativo", Falso_negativo)
                    print("Verdadeiro_positivo", Verdadeiro_positivo)
                    print("Falso_positivo", Falso_positivo)
                    print("Verdadeiro_negativo", Verdadeiro_negativo)  
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=3)
                    #cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    #while True:
                        #if cv2.waitKey(25) & 0xFF == ord('q'):
                            #cv2.destroyAllWindows()
                            #break

    except Exception as e:
        print(e)
        #cap.release()

print("Falso_negativo", Falso_negativo)
print("Verdadeiro_positivo", Verdadeiro_positivo)
print("Falso_positivo", Falso_positivo)
print("Verdadeiro_negativo", Verdadeiro_negativo)  


step: 0
['1outa1.jpg', 2, 0.38201445, 0.4826436, 0.7661095, 0.5972091] 

index
1 [[0 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 1
Falso_positivo 0
Verdadeiro_negativo 0
step: 1
['1outa100.jpg', 2, 0.41235706, 0.4312395, 0.77583134, 0.6160182] 

index
1 [[1 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 2
Falso_positivo 0
Verdadeiro_negativo 0
step: 2
['1outa112.jpg', 2, 0.41708136, 0.4690432, 0.8434385, 0.589644] 

index
1 [[2 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 3
Falso_positivo 0
Verdadeiro_negativo 0
step: 3
['1outa153.jpg', 2, 0.45553687, 0.3830886, 0.70470124, 0.593476] 

index
1 [[3 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 4
Falso_positivo 0
Verdadeiro_negativo 0
step: 4
['1outa167.jpg', 2

['1outa513.jpg', 2, 0.5030744, 0.4426924, 0.7791128, 0.64389277] 

index
1 [[34  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 35
Falso_positivo 0
Verdadeiro_negativo 0
step: 35
['1outa524.jpg', 2, 0.47620624, 0.4260712, 0.79650396, 0.6417004] 

index
1 [[35  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 36
Falso_positivo 0
Verdadeiro_negativo 0
step: 36
['1outa539.jpg', 2, 0.48292655, 0.42648053, 0.8011653, 0.61617124] 

index
1 [[36  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 37
Falso_positivo 0
Verdadeiro_negativo 0
step: 37
['1outa557.jpg', 2, 0.47283524, 0.4117666, 0.73888177, 0.63558644] 

index
1 [[37  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 38
Falso_positivo 0
Verdadeiro_negativo 0
step: 38
['1

['1outa818.jpg', 2, 0.38288584, 0.4501816, 0.71152914, 0.622594] 

index
1 [[67  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 67
Falso_positivo 2
Verdadeiro_negativo 0
step: 68
['1outa840.jpg', 2, 0.33884072, 0.44394752, 0.7429066, 0.56617105] 

index
1 [[68  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 68
Falso_positivo 2
Verdadeiro_negativo 0
step: 69
['1outa841.jpg', 2, 0.35016066, 0.456182, 0.74126023, 0.5382643] 

index
1 [[69  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 69
Falso_positivo 2
Verdadeiro_negativo 0
step: 70
['1outa854.jpg', 2, 0.34701648, 0.4032987, 0.67125607, 0.567657] 

index
1 [[70  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 70
Falso_positivo 2
Verdadeiro_negativo 0
step: 71
['1out

['1outb279.jpg', 1, 0.41801143, 0.39187977, 0.69783306, 0.5854081] 

index
1 [[100   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 100
Falso_positivo 4
Verdadeiro_negativo 0
step: 101
['1outb292.jpg', 1, 0.43969795, 0.41963512, 0.6893864, 0.61036927] 

index
1 [[101   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 101
Falso_positivo 4
Verdadeiro_negativo 0
step: 102
['1outb314.jpg', 1, 0.3713388, 0.47045982, 0.7489755, 0.57851934] 

index
1 [[102   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 102
Falso_positivo 4
Verdadeiro_negativo 0
step: 103
index
1 [[103   0]]
1
arr1 1
Falso_negativo 2
Verdadeiro_positivo 102
Falso_positivo 4
Verdadeiro_negativo 0
step: 104
['1outb35.jpg', 1, 0.39503765, 0.4249518, 0.7427373, 0.62455726] 

['1outb35.jpg', 4, 0.3529538, 0.1801767, 0.6002248, 0

['1outb663.jpg', 1, 0.4122662, 0.5002327, 0.7332366, 0.6773517] 

index
1 [[133   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 3
Verdadeiro_positivo 131
Falso_positivo 7
Verdadeiro_negativo 0
step: 134
['1outb683.jpg', 1, 0.33963186, 0.46420375, 0.71246415, 0.62599164] 

index
1 [[134   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 3
Verdadeiro_positivo 132
Falso_positivo 7
Verdadeiro_negativo 0
step: 135
['1outb684.jpg', 1, 0.34621024, 0.46209276, 0.7016872, 0.6292162] 

index
1 [[135   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 3
Verdadeiro_positivo 133
Falso_positivo 7
Verdadeiro_negativo 0
step: 136
['1outb686.jpg', 1, 0.36538187, 0.4574001, 0.69527113, 0.6265062] 

index
1 [[136   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 3
Verdadeiro_positivo 134
Falso_positivo 7
Verdadeiro_negativo 

...

['1outd143.jpg', 4, 0.4733702, 0.510656, 0.73074114, 0.7474773] 

index
1 [[232   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 227
Falso_positivo 10
Verdadeiro_negativo 0
step: 233
['1outd151.jpg', 4, 0.46348125, 0.5237404, 0.72840005, 0.7851206] 

index
1 [[233   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 228
Falso_positivo 10
Verdadeiro_negativo 0
step: 234
['1outd156.jpg', 4, 0.33912, 0.53707045, 0.70137894, 0.76766413] 

index
1 [[234   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 229
Falso_positivo 10
Verdadeiro_negativo 0
step: 235
['1outd157.jpg', 4, 0.3457975, 0.5423927, 0.70297384, 0.7681119] 

index
1 [[235   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 230
Falso_positivo 10
Verdadeiro_negativo 

['1outd74.jpg', 4, 0.49050358, 0.53932595, 0.7148828, 0.7886541] 

index
1 [[261   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 256
Falso_positivo 20
Verdadeiro_negativo 0
step: 262
['1outd79.jpg', 4, 0.46597862, 0.5230333, 0.7458546, 0.7643723] 

index
1 [[262   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 257
Falso_positivo 20
Verdadeiro_negativo 0
step: 263
['1outd81.jpg', 4, 0.47920614, 0.5266894, 0.7455675, 0.78722095] 

index
1 [[263   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 258
Falso_positivo 20
Verdadeiro_negativo 0
step: 264
['1outd93.jpg', 4, 0.52358806, 0.56300753, 0.819659, 0.84584373] 

index
1 [[264   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 259
Falso_positivo 20
Verdadeiro_negativo 0

step: 298
['1oute354.jpg', 3, 0.06530419, 0.31386438, 0.8088448, 0.76569307] 

index
1 [[298   0]]
1
arr1 3
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 27
Verdadeiro_positivo 272
Falso_positivo 34
Verdadeiro_negativo 0
step: 299
['1oute358.jpg', 5, 0.0, 0.30391738, 0.87661016, 0.8314278] 

['1oute358.jpg', 3, 0.020866722, 0.32085443, 0.8376472, 0.76779664] 

index
1 [[299   0]]
1
arr1 3
k [False, True]
q [[False, True]]
m 2
n 1
F1 0
Coluna 0 True
F1 1
Coluna 1 False
F2 0
Linha 0 True
Falso_negativo 27
Verdadeiro_positivo 273
Falso_positivo 35
Verdadeiro_negativo 0
step: 300
['1oute363.jpg', 3, 0.12349364, 0.29548246, 0.74341905, 0.7121796] 

index
1 [[300   0]]
1
arr1 3
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 27
Verdadeiro_positivo 274
Falso_positivo 35
Verdadeiro_negativo 0
step: 301
['1oute37.jpg', 4, 0.3490708, 0.34345043, 0.9883932, 0.7684605] 

index
1 [[301   0]]
1
arr1 3
k [False]
q [[False]]
m 1
n 1
F

step: 331
index
1 [[331   0]]
1
arr1 2
Falso_negativo 38
Verdadeiro_positivo 294
Falso_positivo 45
Verdadeiro_negativo 0
step: 332
['IMG_8894.JPG', 2, 0.34099886, 0.43670905, 0.8325162, 0.7273303] 

index
1 [[332   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 38
Verdadeiro_positivo 295
Falso_positivo 45
Verdadeiro_negativo 0
step: 333
['IMG_8913.JPG', 2, 0.39201003, 0.47289217, 0.68899435, 0.6632415] 

index
1 [[333   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 38
Verdadeiro_positivo 296
Falso_positivo 45
Verdadeiro_negativo 0
step: 334
['IMG_8929.JPG', 1, 0.17281029, 0.5122065, 0.3603616, 0.66565365] 

['IMG_8929.JPG', 1, 0.41281784, 0.43482396, 0.6175058, 0.54618376] 

index
1 [[334   0]
 [335   0]]
2
arr1 2
k [False, False]
k [False, True]
q [[False, False], [False, True]]
m 2
n 2
F1 0
Coluna 0 True
F1 1
Coluna 1 False
F2 0
Linha 0 False
F2 1
Linha 1 True
Falso_negativo 39
Verdadeiro_p

step: 372
['IMG_9363.JPG', 1, 0.42653877, 0.48298812, 0.584186, 0.5900358] 

index
1 [[373   0]]
1
arr1 0
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 53
Verdadeiro_positivo 302
Falso_positivo 65
Verdadeiro_negativo 19
step: 373
index
1 [[374   0]]
1
arr1 0
Falso_negativo 53
Verdadeiro_positivo 302
Falso_positivo 65
Verdadeiro_negativo 20
step: 374
index
1 [[375   0]]
1
arr1 0
Falso_negativo 53
Verdadeiro_positivo 302
Falso_positivo 65
Verdadeiro_negativo 21
step: 375
['IMG_9366.JPG', 5, 0.2656536, 0.29213476, 0.75373733, 0.5819469] 

index
1 [[376   0]]
1
arr1 0
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 54
Verdadeiro_positivo 302
Falso_positivo 66
Verdadeiro_negativo 21
step: 376
['IMG_9367.JPG', 4, 0.30771524, 0.5193, 0.76618797, 0.7999468] 

['IMG_9367.JPG', 5, 0.3761559, 0.16956003, 0.78953785, 0.46325284] 

index
1 [[377   0]]
1
arr1 0
k [False, False]
q [[False, False]]
m 2
n 1
F1 0
Coluna 0 True
F1 1


In [11]:
presisao = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_positivo)
presisao

0.8162162162162162

In [12]:
seletividade = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_negativo)
seletividade

0.84593837535014

In [13]:
especificidade = Verdadeiro_negativo/(Verdadeiro_negativo+Falso_positivo)
especificidade

0.23595505617977527

In [14]:
acuracidade = (Verdadeiro_positivo+Verdadeiro_negativo)/(Verdadeiro_positivo+Verdadeiro_negativo+Falso_negativo+Falso_positivo)
acuracidade

0.7242152466367713